# Module 9: System Information and Administration

## Welcome to Module 9!

In Module 8, you learned to schedule automation. Now it's time to gather **comprehensive system information** and build monitoring solutions.

### What You'll Learn

- Getting detailed system information
- Monitoring disk usage and health
- Managing processes
- Managing Windows services
- Network information and diagnostics
- User and group management basics
- Generating system health reports
- Building a system health checker

### Why System Information Matters

System monitoring lets you:
- **Detect** problems before they cause failures
- **Track** resource usage over time
- **Optimize** system performance
- **Troubleshoot** issues quickly
- **Report** on system health

Let's become system administrators!

## Setup: Prepare Practice Environment

In [1]:
import subprocess
from pathlib import Path
import json

# Practice folder
practice_folder = Path.home() / "Documents" / "AutomationPractice"
practice_folder.mkdir(exist_ok=True)

# Create SysInfo practice subfolder
sysinfo_practice = practice_folder / "SysInfo_Practice"
sysinfo_practice.mkdir(exist_ok=True)

print(f"Practice folder ready: {sysinfo_practice}")
print("Let's gather system information!\n")

# Helper function
def run_ps(command, cwd=None):
    """Run a PowerShell command and return output."""
    if cwd is None:
        cwd = str(sysinfo_practice)
    
    result = subprocess.run(
        ['powershell', '-Command', command],
        cwd=cwd,
        capture_output=True,
        text=True,
        timeout=60
    )
    
    return result.stdout + result.stderr

print("✓ Helper function ready!")

Practice folder ready: C:\Users\USER\Documents\AutomationPractice\SysInfo_Practice
Let's gather system information!

✓ Helper function ready!


## 1. System Information

Gather comprehensive information about the computer.

### Basic Computer Information

In [2]:
output = run_ps('''
Write-Host "=== Computer Information ==="
Write-Host ""

# Get computer info
$computerInfo = Get-ComputerInfo

Write-Host "Computer Name: $($computerInfo.CsName)"
Write-Host "Manufacturer: $($computerInfo.CsManufacturer)"
Write-Host "Model: $($computerInfo.CsModel)"
Write-Host ""

Write-Host "Operating System:"
Write-Host "  OS Name: $($computerInfo.OsName)"
Write-Host "  Version: $($computerInfo.OsVersion)"
Write-Host "  Build: $($computerInfo.OsBuildNumber)"
Write-Host "  Architecture: $($computerInfo.OsArchitecture)"
Write-Host ""

Write-Host "Hardware:"
Write-Host "  Processors: $($computerInfo.CsNumberOfProcessors)"
Write-Host "  Logical Processors: $($computerInfo.CsNumberOfLogicalProcessors)"
Write-Host "  Total RAM: $([math]::Round($computerInfo.CsTotalPhysicalMemory / 1GB, 2)) GB"
Write-Host ""

Write-Host "System:"
Write-Host "  Last Boot: $($computerInfo.OsLastBootUpTime)"
Write-Host "  Uptime: $((Get-Date) - $computerInfo.OsLastBootUpTime | Select-Object -ExpandProperty Days) days"
Write-Host "  Time Zone: $($computerInfo.TimeZone)"
''')

print(output)

=== Computer Information ===

Computer Name: LAPTOP-D9EUFCH4
Manufacturer: ASUSTeK COMPUTER INC.
Model: ASUS TUF Gaming A15 FA507NVR_FA507NVR

Operating System:
  OS Name: Microsoft Windows 11 Home Single Language
  Version: 10.0.26200
  Build: 26200
  Architecture: 64-bit

Hardware:
  Processors: 1
  Logical Processors: 16
  Total RAM: 31.82 GB

System:
  Last Boot: 11/14/2025 10:52:43
  Uptime: 0 days
  Time Zone: (UTC+08:00) Kuala Lumpur, Singapore



### CPU and Memory Information

In [3]:
output = run_ps('''
Write-Host "=== CPU and Memory Information ==="
Write-Host ""

# CPU Information
$cpu = Get-WmiObject Win32_Processor | Select-Object -First 1

Write-Host "CPU:"
Write-Host "  Name: $($cpu.Name)"
Write-Host "  Cores: $($cpu.NumberOfCores)"
Write-Host "  Logical Processors: $($cpu.NumberOfLogicalProcessors)"
Write-Host "  Max Speed: $($cpu.MaxClockSpeed) MHz"
Write-Host ""

# Memory Information
$memory = Get-WmiObject Win32_OperatingSystem
$totalMemory = [math]::Round($memory.TotalVisibleMemorySize / 1MB, 2)
$freeMemory = [math]::Round($memory.FreePhysicalMemory / 1MB, 2)
$usedMemory = $totalMemory - $freeMemory
$memoryPercent = [math]::Round(($usedMemory / $totalMemory) * 100, 1)

Write-Host "Memory:"
Write-Host "  Total: $totalMemory GB"
Write-Host "  Used: $usedMemory GB ($memoryPercent%)"
Write-Host "  Free: $freeMemory GB"
''')

print(output)

=== CPU and Memory Information ===

CPU:
  Name: AMD Ryzen 7 7435HS                             
  Cores: 8
  Logical Processors: 16
  Max Speed: 3100 MHz

Memory:
  Total: 31.82 GB
  Used: 25.07 GB (78.8%)
  Free: 6.75 GB



## 2. Disk Information and Monitoring

Monitor disk usage and health.

### Disk Usage

In [4]:
output = run_ps('''
Write-Host "=== Disk Usage ==="
Write-Host ""

$disks = Get-PSDrive -PSProvider FileSystem | Where-Object {$_.Used -ne $null}

foreach ($disk in $disks) {
    $totalSize = [math]::Round(($disk.Used + $disk.Free) / 1GB, 2)
    $usedSize = [math]::Round($disk.Used / 1GB, 2)
    $freeSize = [math]::Round($disk.Free / 1GB, 2)
    $percentFree = [math]::Round(($disk.Free / ($disk.Used + $disk.Free)) * 100, 1)
    
    Write-Host "Drive $($disk.Name):"
    Write-Host "  Total: $totalSize GB"
    Write-Host "  Used: $usedSize GB"
    Write-Host "  Free: $freeSize GB ($percentFree% free)"
    
    if ($percentFree -lt 10) {
        Write-Host "  WARNING: Low disk space!" -ForegroundColor Red
    }
    elseif ($percentFree -lt 20) {
        Write-Host "  CAUTION: Disk space running low" -ForegroundColor Yellow
    }
    else {
        Write-Host "  Status: OK" -ForegroundColor Green
    }
    Write-Host ""
}
''')

print(output)

=== Disk Usage ===

Drive C:
  Total: 447.39 GB
  Used: 379.98 GB
  Free: 67.41 GB (15.1% free)
  CAUTION: Disk space running low

Drive D:
  Total: 1863 GB
  Used: 506.83 GB
  Free: 1356.17 GB (72.8% free)
  Status: OK




### Detailed Disk Information

In [5]:
output = run_ps('''
Write-Host "=== Detailed Disk Information ==="
Write-Host ""

$volumes = Get-Volume | Where-Object {$_.DriveLetter -ne $null}

foreach ($vol in $volumes) {
    Write-Host "Volume $($vol.DriveLetter):"
    Write-Host "  Label: $($vol.FileSystemLabel)"
    Write-Host "  File System: $($vol.FileSystem)"
    Write-Host "  Health Status: $($vol.HealthStatus)"
    Write-Host "  Operational Status: $($vol.OperationalStatus)"
    Write-Host ""
}
''')

print(output)

=== Detailed Disk Information ===

Volume D:
  Label: Data
  File System: NTFS
  Health Status: Healthy
  Operational Status: OK

Volume C:
  Label: OS
  File System: NTFS
  Health Status: Healthy
  Operational Status: OK




## 3. Process Management

View and manage running processes.

### Viewing Processes

In [6]:
output = run_ps('''
Write-Host "=== Top 10 Processes by Memory Usage ==="
Write-Host ""

$processes = Get-Process | Sort-Object WorkingSet -Descending | Select-Object -First 10

Write-Host "Rank  Process Name          PID     Memory (MB)  CPU (s)"
Write-Host "----  -------------------   ------  -----------  -------"

$rank = 1
foreach ($proc in $processes) {
    $memoryMB = [math]::Round($proc.WorkingSet / 1MB, 2)
    $cpuTime = [math]::Round($proc.TotalProcessorTime.TotalSeconds, 1)
    $name = $proc.ProcessName.PadRight(20).Substring(0, 20)
    
    Write-Host ("{0,-5} {1,-20} {2,6}  {3,11}  {4,7}" -f $rank, $name, $proc.Id, $memoryMB, $cpuTime)
    $rank++
}
''')

print(output)

=== Top 10 Processes by Memory Usage ===

Rank  Process Name          PID     Memory (MB)  CPU (s)
----  -------------------   ------  -----------  -------
1     vmmemWSL              33032      1966.25        0
2     Code                   4016      1354.01    616.6
3     Code                   8448       680.98   1317.4
4     Code                   3400       678.85    336.2
5     Code                  25228       653.16    847.8
6     node                  34664       570.78   1287.9
7     Code                  20824       481.62   4252.4
8     chrome                 2100       409.93    715.8
9     mc-fw-host             6592       392.46        0
10    brave                 31520       375.43    132.1



### Process Statistics

In [7]:
output = run_ps('''
Write-Host "=== Process Statistics ==="
Write-Host ""

$allProcesses = Get-Process
$totalProcesses = $allProcesses.Count
$totalMemory = ($allProcesses | Measure-Object WorkingSet -Sum).Sum
$totalMemoryGB = [math]::Round($totalMemory / 1GB, 2)

Write-Host "Total Running Processes: $totalProcesses"
Write-Host "Total Memory Used: $totalMemoryGB GB"
Write-Host ""

# Count processes by company
$byCompany = $allProcesses | Where-Object {$_.Company} | Group-Object Company | Sort-Object Count -Descending | Select-Object -First 5

Write-Host "Top 5 Companies by Process Count:"
foreach ($company in $byCompany) {
    Write-Host "  $($company.Name): $($company.Count) processes"
}
''')

print(output)

=== Process Statistics ===

Total Running Processes: 472
Total Memory Used: 27.37 GB

Top 5 Companies by Process Count:
  Microsoft Corporation: 152 processes
  Google LLC: 50 processes
  Brave Software, Inc.: 16 processes
  Python Software Foundation: 15 processes
  Node.js: 9 processes



### Starting and Stopping Processes

In [8]:
output = run_ps('''
Write-Host "=== Process Management Example ==="
Write-Host ""

# Start notepad as example
Write-Host "Starting notepad..."
$process = Start-Process notepad -PassThru

Write-Host "Process started:"
Write-Host "  Name: $($process.ProcessName)"
Write-Host "  PID: $($process.Id)"
Write-Host "  Start Time: $($process.StartTime)"
Write-Host ""

# Wait a moment
Start-Sleep -Seconds 1

# Check if running
$running = Get-Process -Id $process.Id -ErrorAction SilentlyContinue
if ($running) {
    Write-Host "Process is running" -ForegroundColor Green
    Write-Host "  Memory: $([math]::Round($running.WorkingSet / 1MB, 2)) MB"
}

# Stop the process
Write-Host ""
Write-Host "Stopping notepad..."
Stop-Process -Id $process.Id -Force

Start-Sleep -Seconds 1

# Verify stopped
$stopped = Get-Process -Id $process.Id -ErrorAction SilentlyContinue
if (-not $stopped) {
    Write-Host "Process stopped successfully" -ForegroundColor Green
}
''')

print(output)

=== Process Management Example ===

Starting notepad...
Process started:
  Name: 
  PID: 
  Start Time: 


Stopping notepad...
Process stopped successfully
Start-Process : This command cannot be run completely because the system cannot find all the information required.
At line:7 char:12
+ $process = Start-Process notepad -PassThru
+            ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : InvalidOperation: (:) [Start-Process], InvalidOperationException
    + FullyQualifiedErrorId : InvalidOperationException,Microsoft.PowerShell.Commands.StartProcessCommand
 
Get-Process : Cannot bind argument to parameter 'Id' because it is null.
At line:19 char:28
+ $running = Get-Process -Id $process.Id -ErrorAction SilentlyContinue
+                            ~~~~~~~~~~~
    + CategoryInfo          : InvalidData: (:) [Get-Process], ParameterBindingValidationException
    + FullyQualifiedErrorId : ParameterArgumentValidationErrorNullNotAllowed,Microsoft.PowerShell.Commands.GetProces

## 4. Service Management

Manage Windows services.

### Viewing Services

In [9]:
output = run_ps('''
Write-Host "=== Windows Services Overview ==="
Write-Host ""

$services = Get-Service

$running = ($services | Where-Object {$_.Status -eq 'Running'}).Count
$stopped = ($services | Where-Object {$_.Status -eq 'Stopped'}).Count
$total = $services.Count

Write-Host "Service Status Summary:"
Write-Host "  Total Services: $total"
Write-Host "  Running: $running"
Write-Host "  Stopped: $stopped"
Write-Host ""

Write-Host "Sample Running Services (first 10):"
$runningServices = Get-Service | Where-Object {$_.Status -eq 'Running'} | Select-Object -First 10

foreach ($svc in $runningServices) {
    Write-Host "  $($svc.Name) - $($svc.DisplayName)"
}
''')

print(output)

=== Windows Services Overview ===

Service Status Summary:
  Total Services: 316
  Running: 143
  Stopped: 173

Sample Running Services (first 10):
  Appinfo - Application Information
  AppXSvc - AppX Deployment Service (AppXSVC)
  ArmouryCrateControlInterface - Armoury Crate Control Interface
  ArmouryCrateService - Armoury Crate Service
  AsusAppService - ASUS App Service
  AsusCertService - Asus Certificate Service
  ASUSOptimization - ASUS Optimization
  AsusPTPService - AsusPTPService
  ASUSSoftwareManager - ASUS Software Manager
  ASUSSwitch - ASUS Switch



### Service Details

In [10]:
output = run_ps('''
Write-Host "=== Service Details Example ==="
Write-Host ""

# Get a common service
$serviceName = "Winmgmt"  # Windows Management Instrumentation
$service = Get-Service $serviceName

Write-Host "Service: $($service.DisplayName)"
Write-Host "  Name: $($service.Name)"
Write-Host "  Status: $($service.Status)"
Write-Host "  Start Type: $($service.StartType)"
Write-Host ""

# Get dependent services
$dependents = $service.DependentServices
if ($dependents.Count -gt 0) {
    Write-Host "Dependent Services: $($dependents.Count)"
    $dependents | Select-Object -First 5 | ForEach-Object {
        Write-Host "  - $($_.DisplayName) ($($_.Status))"
    }
}
else {
    Write-Host "No dependent services"
}
''')

print(output)

=== Service Details Example ===

Service: Windows Management Instrumentation
  Name: Winmgmt
  Status: Running
  Start Type: Automatic

No dependent services



## 5. Network Information

Gather network configuration and diagnostics.

### Network Adapters

In [11]:
output = run_ps('''
Write-Host "=== Network Adapters ==="
Write-Host ""

$adapters = Get-NetAdapter | Where-Object {$_.Status -eq 'Up'}

foreach ($adapter in $adapters) {
    Write-Host "Adapter: $($adapter.Name)"
    Write-Host "  Description: $($adapter.InterfaceDescription)"
    Write-Host "  Status: $($adapter.Status)"
    Write-Host "  Link Speed: $($adapter.LinkSpeed)"
    Write-Host "  MAC Address: $($adapter.MacAddress)"
    
    # Get IP configuration
    $ipConfig = Get-NetIPAddress -InterfaceIndex $adapter.ifIndex -ErrorAction SilentlyContinue
    $ipv4 = $ipConfig | Where-Object {$_.AddressFamily -eq 'IPv4'}
    
    if ($ipv4) {
        Write-Host "  IPv4 Address: $($ipv4.IPAddress)"
        Write-Host "  Prefix Length: $($ipv4.PrefixLength)"
    }
    
    Write-Host ""
}
''')

print(output)

=== Network Adapters ===

Adapter: Ethernet 3
  Description: VirtualBox Host-Only Ethernet Adapter
  Status: Up
  Link Speed: 1 Gbps
  MAC Address: 0A-00-27-00-00-12
  IPv4 Address: 192.168.56.1
  Prefix Length: 24

Adapter: Wi-Fi
  Description: Realtek 8852BE Wireless LAN WiFi 6 PCI-E NIC
  Status: Up
  Link Speed: 144.4 Mbps
  MAC Address: 94-BB-43-EA-BE-06
  IPv4 Address: 192.168.1.3
  Prefix Length: 24

Adapter: vEthernet (WSL (Hyper-V firewall))
  Description: Hyper-V Virtual Ethernet Adapter
  Status: Up
  Link Speed: 10 Gbps
  MAC Address: 00-15-5D-33-40-C5
  IPv4 Address: 172.31.160.1
  Prefix Length: 20




### Network Statistics

In [12]:
output = run_ps('''
Write-Host "=== Network Statistics ==="
Write-Host ""

$stats = Get-NetAdapterStatistics | Where-Object {$_.Name -notlike '*Loopback*'} | Select-Object -First 3

foreach ($stat in $stats) {
    $receivedGB = [math]::Round($stat.ReceivedBytes / 1GB, 2)
    $sentGB = [math]::Round($stat.SentBytes / 1GB, 2)
    
    Write-Host "Adapter: $($stat.Name)"
    Write-Host "  Received: $receivedGB GB"
    Write-Host "  Sent: $sentGB GB"
    Write-Host "  Total: $([math]::Round($receivedGB + $sentGB, 2)) GB"
    Write-Host ""
}
''')

print(output)

=== Network Statistics ===

Adapter: Ethernet
  Received: 0 GB
  Sent: 0 GB
  Total: 0 GB

Adapter: Ethernet 3
  Received: 0 GB
  Sent: 0 GB
  Total: 0 GB

Adapter: Wi-Fi
  Received: 0.06 GB
  Sent: 0.02 GB
  Total: 0.08 GB




### Connection Testing

In [13]:
output = run_ps('''
Write-Host "=== Network Connectivity Test ==="
Write-Host ""

# Test common connections
$targets = @(
    @{Name="Loopback"; Address="127.0.0.1"},
    @{Name="Google DNS"; Address="8.8.8.8"},
    @{Name="Cloudflare DNS"; Address="1.1.1.1"}
)

foreach ($target in $targets) {
    Write-Host "Testing: $($target.Name) ($($target.Address))"
    
    $result = Test-Connection -ComputerName $target.Address -Count 2 -Quiet
    
    if ($result) {
        Write-Host "  Status: Reachable" -ForegroundColor Green
        
        # Get latency
        $ping = Test-Connection -ComputerName $target.Address -Count 1
        Write-Host "  Latency: $($ping.ResponseTime) ms"
    }
    else {
        Write-Host "  Status: Not Reachable" -ForegroundColor Red
    }
    Write-Host ""
}
''')

print(output)

=== Network Connectivity Test ===

Testing: Loopback (127.0.0.1)
  Status: Reachable
  Latency: 0 ms

Testing: Google DNS (8.8.8.8)
  Status: Reachable
  Latency: 14 ms

Testing: Cloudflare DNS (1.1.1.1)
  Status: Reachable
  Latency: 14 ms




## 6. User and Group Information

View user and group information (basic).

### Current User Information

In [14]:
output = run_ps('''
Write-Host "=== Current User Information ==="
Write-Host ""

Write-Host "Username: $env:USERNAME"
Write-Host "Domain: $env:USERDOMAIN"
Write-Host "User Profile: $env:USERPROFILE"
Write-Host "Computer Name: $env:COMPUTERNAME"
Write-Host ""

# Check admin privileges
$currentUser = [Security.Principal.WindowsIdentity]::GetCurrent()
$principal = New-Object Security.Principal.WindowsPrincipal($currentUser)
$isAdmin = $principal.IsInRole([Security.Principal.WindowsBuiltInRole]::Administrator)

if ($isAdmin) {
    Write-Host "Running as: Administrator" -ForegroundColor Yellow
}
else {
    Write-Host "Running as: Standard User"
}
''')

print(output)

=== Current User Information ===

Username: USER
Domain: LAPTOP-D9EUFCH4
User Profile: C:\Users\USER
Computer Name: LAPTOP-D9EUFCH4

Running as: Standard User



### Local Users and Groups

In [15]:
output = run_ps('''
Write-Host "=== Local Users and Groups ==="
Write-Host ""

try {
    # Get local users
    $users = Get-LocalUser
    
    Write-Host "Local Users: $($users.Count)"
    Write-Host ""
    
    Write-Host "User Accounts (first 5):"
    $users | Select-Object -First 5 | ForEach-Object {
        Write-Host "  $($_.Name)"
        Write-Host "    Enabled: $($_.Enabled)"
        Write-Host "    Last Logon: $($_.LastLogon)"
        Write-Host ""
    }
    
    # Get local groups
    $groups = Get-LocalGroup
    
    Write-Host "Local Groups: $($groups.Count)"
    Write-Host ""
    
    Write-Host "Sample Groups (first 5):"
    $groups | Select-Object -First 5 | ForEach-Object {
        Write-Host "  - $($_.Name): $($_.Description)"
    }
}
catch {
    Write-Host "Note: Get-LocalUser/Get-LocalGroup may require administrator privileges" -ForegroundColor Yellow
    Write-Host "Error: $($_.Exception.Message)"
}
''')

print(output)

=== Local Users and Groups ===

Local Users: 5

User Accounts (first 5):
  Administrator
    Enabled: False
    Last Logon: 07/07/2024 13:20:19

  DefaultAccount
    Enabled: False
    Last Logon: 

  Guest
    Enabled: False
    Last Logon: 

  USER
    Enabled: True
    Last Logon: 11/15/2025 08:14:49

  WDAGUtilityAccount
    Enabled: False
    Last Logon: 

Local Groups: 16

Sample Groups (first 5):
  - docker-users: Members of this group can access features exposed by the Docker Desktop service (such as Windows Containers and the Hyper-V backend).
  - ORA_DBA: Oracle DBA Group
  - Administrators: Administrators have complete and unrestricted access to the computer/domain
  - Device Owners: Members of this group can change system-wide settings.
  - Distributed COM Users: Members are allowed to launch, activate and use Distributed COM objects on this machine.



## 7. Practical Project: System Health Checker

Build a comprehensive system health monitoring tool.

### Create Health Checker Script

In [16]:
# Create comprehensive health checker
health_checker = '''
# System Health Checker
# Generates comprehensive system health report

param(
    [string]$OutputPath = "SystemHealth.html",
    [switch]$OpenReport
)

$reportData = @{}
$issues = @()

function Add-Issue {
    param($Severity, $Message)
    $script:issues += @{Severity=$Severity; Message=$Message}
}

Write-Host "=== System Health Check Started ==="
Write-Host ""

# 1. System Information
Write-Host "[1/6] Gathering system information..."
$computerInfo = Get-ComputerInfo
$reportData.ComputerName = $computerInfo.CsName
$reportData.OS = $computerInfo.OsName
$reportData.OSVersion = $computerInfo.OsVersion
$reportData.LastBoot = $computerInfo.OsLastBootUpTime
$reportData.Uptime = ((Get-Date) - $computerInfo.OsLastBootUpTime).Days

# 2. Memory Check
Write-Host "[2/6] Checking memory usage..."
$memory = Get-WmiObject Win32_OperatingSystem
$totalMemoryGB = [math]::Round($memory.TotalVisibleMemorySize / 1MB, 2)
$freeMemoryGB = [math]::Round($memory.FreePhysicalMemory / 1MB, 2)
$usedMemoryGB = $totalMemoryGB - $freeMemoryGB
$memoryPercent = [math]::Round(($usedMemoryGB / $totalMemoryGB) * 100, 1)

$reportData.TotalMemory = $totalMemoryGB
$reportData.UsedMemory = $usedMemoryGB
$reportData.MemoryPercent = $memoryPercent

if ($memoryPercent -gt 90) {
    Add-Issue "Critical" "Memory usage critical: ${memoryPercent}%"
}
elseif ($memoryPercent -gt 80) {
    Add-Issue "Warning" "Memory usage high: ${memoryPercent}%"
}

# 3. Disk Check
Write-Host "[3/6] Checking disk space..."
$disks = Get-PSDrive -PSProvider FileSystem | Where-Object {$_.Used -ne $null}
$diskInfo = @()

foreach ($disk in $disks) {
    $totalGB = [math]::Round(($disk.Used + $disk.Free) / 1GB, 2)
    $freeGB = [math]::Round($disk.Free / 1GB, 2)
    $percentFree = [math]::Round(($disk.Free / ($disk.Used + $disk.Free)) * 100, 1)
    
    $diskInfo += @{
        Name = $disk.Name
        Total = $totalGB
        Free = $freeGB
        PercentFree = $percentFree
    }
    
    if ($percentFree -lt 10) {
        Add-Issue "Critical" "Drive ${disk.Name}: Low disk space (${percentFree}% free)"
    }
    elseif ($percentFree -lt 20) {
        Add-Issue "Warning" "Drive ${disk.Name}: Disk space running low (${percentFree}% free)"
    }
}

$reportData.Disks = $diskInfo

# 4. Process Check
Write-Host "[4/6] Analyzing processes..."
$processes = Get-Process
$topProcesses = $processes | Sort-Object WorkingSet -Descending | Select-Object -First 5
$reportData.TotalProcesses = $processes.Count
$reportData.TopProcesses = $topProcesses | ForEach-Object {
    @{
        Name = $_.ProcessName
        Memory = [math]::Round($_.WorkingSet / 1MB, 2)
    }
}

# 5. Service Check
Write-Host "[5/6] Checking services..."
$services = Get-Service
$runningServices = ($services | Where-Object {$_.Status -eq ''Running''}).Count
$stoppedServices = ($services | Where-Object {$_.Status -eq ''Stopped''}).Count

$reportData.TotalServices = $services.Count
$reportData.RunningServices = $runningServices
$reportData.StoppedServices = $stoppedServices

# 6. Network Check
Write-Host "[6/6] Testing network connectivity..."
$networkOK = Test-Connection -ComputerName "8.8.8.8" -Count 1 -Quiet
$reportData.NetworkStatus = if ($networkOK) {"Connected"} else {"Disconnected"}

if (-not $networkOK) {
    Add-Issue "Warning" "No internet connectivity detected"
}

# Generate HTML Report
Write-Host ""
Write-Host "Generating report..."

$html = @"
<!DOCTYPE html>
<html>
<head>
    <title>System Health Report</title>
    <style>
        body { font-family: Arial, sans-serif; margin: 20px; background: #f5f5f5; }
        h1 { color: #333; }
        .section { background: white; padding: 20px; margin: 10px 0; border-radius: 5px; box-shadow: 0 2px 5px rgba(0,0,0,0.1); }
        .metric { display: inline-block; margin: 10px 20px 10px 0; }
        .label { font-weight: bold; color: #666; }
        .value { font-size: 1.2em; color: #333; }
        .issue-critical { color: #d32f2f; padding: 10px; background: #ffebee; border-left: 4px solid #d32f2f; margin: 5px 0; }
        .issue-warning { color: #f57c00; padding: 10px; background: #fff3e0; border-left: 4px solid #f57c00; margin: 5px 0; }
        .ok { color: #4caf50; }
        table { width: 100%; border-collapse: collapse; }
        th, td { padding: 10px; text-align: left; border-bottom: 1px solid #ddd; }
        th { background: #f5f5f5; }
    </style>
</head>
<body>
    <h1>System Health Report</h1>
    <div class="section">
        <h2>System Information</h2>
        <div class="metric"><span class="label">Computer:</span> <span class="value">$($reportData.ComputerName)</span></div>
        <div class="metric"><span class="label">OS:</span> <span class="value">$($reportData.OS)</span></div>
        <div class="metric"><span class="label">Uptime:</span> <span class="value">$($reportData.Uptime) days</span></div>
    </div>
    
    <div class="section">
        <h2>Issues Found</h2>
"@

if ($issues.Count -eq 0) {
    $html += "        <p class='ok'>No issues detected - System is healthy!</p>`n"
}
else {
    foreach ($issue in $issues) {
        $class = if ($issue.Severity -eq "Critical") {"issue-critical"} else {"issue-warning"}
        $html += "        <div class='$class'><strong>$($issue.Severity):</strong> $($issue.Message)</div>`n"
    }
}

$html += @"
    </div>
    
    <div class="section">
        <h2>Memory</h2>
        <div class="metric"><span class="label">Total:</span> <span class="value">$($reportData.TotalMemory) GB</span></div>
        <div class="metric"><span class="label">Used:</span> <span class="value">$($reportData.UsedMemory) GB ($($reportData.MemoryPercent)%)</span></div>
    </div>
    
    <div class="section">
        <h2>Disk Space</h2>
        <table>
            <tr><th>Drive</th><th>Total (GB)</th><th>Free (GB)</th><th>Free (%)</th></tr>
"@

foreach ($disk in $reportData.Disks) {
    $html += "            <tr><td>$($disk.Name):</td><td>$($disk.Total)</td><td>$($disk.Free)</td><td>$($disk.PercentFree)%</td></tr>`n"
}

$html += @"
        </table>
    </div>
    
    <div class="section">
        <h2>Services</h2>
        <div class="metric"><span class="label">Total:</span> <span class="value">$($reportData.TotalServices)</span></div>
        <div class="metric"><span class="label">Running:</span> <span class="value">$($reportData.RunningServices)</span></div>
        <div class="metric"><span class="label">Stopped:</span> <span class="value">$($reportData.StoppedServices)</span></div>
    </div>
    
    <div class="section">
        <h2>Network</h2>
        <div class="metric"><span class="label">Status:</span> <span class="value">$($reportData.NetworkStatus)</span></div>
    </div>
    
    <p><small>Generated: $(Get-Date -Format "yyyy-MM-dd HH:mm:ss")</small></p>
</body>
</html>
"@

# Save report
$html | Out-File -FilePath $OutputPath -Encoding UTF8

Write-Host ""
Write-Host "=== Health Check Complete ===" -ForegroundColor Green
Write-Host "Report saved to: $OutputPath"
Write-Host "Issues found: $($issues.Count)"

if ($OpenReport) {
    Start-Process $OutputPath
}
'''

(sysinfo_practice / "system_health_checker.ps1").write_text(health_checker, encoding='utf-8')

print(f"Health checker script created: {sysinfo_practice / 'system_health_checker.ps1'}")
print("\nThis script:")
print("  - Checks system, memory, disk, processes, services, and network")
print("  - Identifies issues and rates severity")
print("  - Generates professional HTML report")
print("  - Can open report automatically")

Health checker script created: C:\Users\USER\Documents\AutomationPractice\SysInfo_Practice\system_health_checker.ps1

This script:
  - Checks system, memory, disk, processes, services, and network
  - Identifies issues and rates severity
  - Generates professional HTML report
  - Can open report automatically


### Run the Health Checker

In [17]:
output = run_ps('''
$scriptPath = Join-Path (Get-Location) "system_health_checker.ps1"
$reportPath = Join-Path (Get-Location) "SystemHealth.html"

Write-Host "Running System Health Checker..."
Write-Host ""

# Run the health checker
& $scriptPath -OutputPath $reportPath

Write-Host ""
Write-Host "Health report generated!"
Write-Host "Location: $reportPath"
''', cwd=str(sysinfo_practice))

print(output)

Running System Health Checker...


Health report generated!
Location: C:\Users\USER\Documents\AutomationPractice\SysInfo_Practice\SystemHealth.html
& : File C:\Users\USER\Documents\AutomationPractice\SysInfo_Practice\system_health_checker.ps1 cannot be loaded 
because running scripts is disabled on this system. For more information, see about_Execution_Policies at 
https:/go.microsoft.com/fwlink/?LinkID=135170.
At line:9 char:3
+ & $scriptPath -OutputPath $reportPath
+   ~~~~~~~~~~~
    + CategoryInfo          : SecurityError: (:) [], PSSecurityException
    + FullyQualifiedErrorId : UnauthorizedAccess



## Summary and Key Takeaways

Congratulations! You've mastered system information gathering and monitoring.

### What You Learned:

✓ **System Information** - Computer, OS, hardware details  
✓ **CPU and Memory** - Resource usage monitoring  
✓ **Disk Monitoring** - Usage, health, and alerts  
✓ **Process Management** - Viewing, starting, stopping processes  
✓ **Service Management** - Managing Windows services  
✓ **Network Information** - Adapters, statistics, connectivity  
✓ **User Management** - User and group information  
✓ **Health Reporting** - Comprehensive system monitoring  

### Quick Reference: System Information

```powershell
# System Information
Get-ComputerInfo
Get-WmiObject Win32_OperatingSystem
Get-WmiObject Win32_Processor

# Disk Information
Get-PSDrive
Get-Volume

# Process Management
Get-Process
Get-Process -Name notepad
Start-Process notepad
Stop-Process -Name notepad

# Service Management
Get-Service
Get-Service -Name Winmgmt
Start-Service -Name ServiceName
Stop-Service -Name ServiceName

# Network Information
Get-NetAdapter
Get-NetIPAddress
Test-Connection -ComputerName 8.8.8.8

# User Information
Get-LocalUser
Get-LocalGroup
```

### Best Practices

1. **Monitor regularly** to catch issues early
2. **Set thresholds** for alerts (disk < 20%, memory > 80%)
3. **Log historical data** for trend analysis
4. **Automate reports** using scheduled tasks
5. **Test notifications** before deployment
6. **Document baselines** for your systems
7. **Use error handling** for robust monitoring
8. **Generate visual reports** for stakeholders
9. **Include timestamps** in all logs
10. **Archive old reports** periodically

### Real-World Applications

- Daily system health checks
- Resource usage monitoring
- Capacity planning
- Performance troubleshooting
- Compliance reporting
- Inventory management
- Alerting and notifications
- Automated diagnostics
- SLA monitoring
- Audit trails

### Next Steps

In **Module 10: Final Capstone Project**, you'll:
- Combine all skills learned
- Build a comprehensive automation solution
- Choose from multiple project options
- Create production-ready code
- Document your work professionally

You're now ready for the final project!

## Cleanup

Run this cell to remove all practice files:

In [18]:
import shutil

print("Cleaning up SysInfo_Practice folder...\n")

if sysinfo_practice.exists():
    shutil.rmtree(sysinfo_practice)
    print(f"✓ Removed {sysinfo_practice}")
    print("\nAll practice files deleted.")
    print("The main AutomationPractice folder remains for the final project.")
else:
    print("Practice folder already cleaned up!")

Cleaning up SysInfo_Practice folder...

✓ Removed C:\Users\USER\Documents\AutomationPractice\SysInfo_Practice

All practice files deleted.
The main AutomationPractice folder remains for the final project.
